In [1]:
from google.colab import files
uploaded = files.upload()

Saving company-document-text.csv to company-document-text (1).csv


In [2]:
import pandas as pd
df=pd.read_csv('company-document-text.csv')

In [3]:
df.head()

,text,label,word_count
0,order id 10718 shipping details ship name k...,ShippingOrder,120
1,invoice order id 10707 customer id arout ord...,invoice,66
2,order id 10448 shipping details ship name r...,ShippingOrder,96
3,invoice order id 11068 customer id queen ord...,invoice,68
4,order id 10656 shipping details ship name g...,ShippingOrder,109


In [4]:
df.isnull().sum()

,0
text,0
label,0
word_count,0


In [5]:
df.duplicated().sum()

np.int64(0)

In [6]:
import nltk
# Force redownload of core tokenizer files
nltk.download('stopwords', force=True)


[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


True

In [7]:
import re
from nltk.corpus import stopwords
from nltk.stem import PorterStemmer

stop_words=set(stopwords.words('english'))
stemmer=PorterStemmer()

In [8]:
# Text cleaning function
def clean_text(text):
    text = text.lower()  # Convert to lowercase
    text = re.sub(r"[^\w\s]", "", text)  # Remove special characters
    text = re.sub(r"\s+", " ", text)  # Remove extra whitespace
    text = " ".join([stemmer.stem(word) for word in text.split() if word not in stop_words])  # Remove stopwords and stem
    return text


In [9]:
df["Cleaned Text"] = df["text"].apply(clean_text)
df

,text,label,word_count,Cleaned Text
0,order id 10718 shipping details ship name k...,ShippingOrder,120,order id 10718 ship detail ship name königlich...
1,invoice order id 10707 customer id arout ord...,invoice,66,invoic order id 10707 custom id arout order da...
2,order id 10448 shipping details ship name r...,ShippingOrder,96,order id 10448 ship detail ship name rancho gr...
3,invoice order id 11068 customer id queen ord...,invoice,68,invoic order id 11068 custom id queen order da...
4,order id 10656 shipping details ship name g...,ShippingOrder,109,order id 10656 ship detail ship name great lak...
...,...,...,...,...
2671,order id 10326 shipping details ship name b...,ShippingOrder,111,order id 10326 ship detail ship name bólido co...
2672,purchase orders order id order date customer n...,purchase Order,39,purchas order order id order date custom name ...
2673,invoice order id 10460 customer id folko ord...,invoice,59,invoic order id 10460 custom id folko order da...
2674,stock report for 2018-01 category meat poult...,report,46,stock report 201801 categori meat poultri id c...


In [10]:
from sklearn.preprocessing import LabelEncoder
label_encoder=LabelEncoder()

df['label']=label_encoder.fit_transform(df['label'])

In [11]:
df

,text,label,word_count,Cleaned Text
0,order id 10718 shipping details ship name k...,0,120,order id 10718 ship detail ship name königlich...
1,invoice order id 10707 customer id arout ord...,1,66,invoic order id 10707 custom id arout order da...
2,order id 10448 shipping details ship name r...,0,96,order id 10448 ship detail ship name rancho gr...
3,invoice order id 11068 customer id queen ord...,1,68,invoic order id 11068 custom id queen order da...
4,order id 10656 shipping details ship name g...,0,109,order id 10656 ship detail ship name great lak...
...,...,...,...,...
2671,order id 10326 shipping details ship name b...,0,111,order id 10326 ship detail ship name bólido co...
2672,purchase orders order id order date customer n...,2,39,purchas order order id order date custom name ...
2673,invoice order id 10460 customer id folko ord...,1,59,invoic order id 10460 custom id folko order da...
2674,stock report for 2018-01 category meat poult...,3,46,stock report 201801 categori meat poultri id c...


In [12]:
from datasets import Dataset
dataset = Dataset.from_pandas(df[['Cleaned Text','label']])

In [13]:
dataset

Dataset({
    features: ['Cleaned Text', 'label'],
    num_rows: 2676
})

In [14]:
# Split the dataset using Hugging Face's train_test_split
split_dataset = dataset.train_test_split(test_size=0.2, seed=42)

# Access train and test splits
train_texts = split_dataset['train']
test_texts = split_dataset['test']

In [15]:
from transformers import AutoTokenizer

# Initialize tokenizer
model_name = "bert-base-uncased"
tokenizer = AutoTokenizer.from_pretrained(model_name)

# Define tokenization function
def tokenize_function(examples):
    return tokenizer(examples["Cleaned Text"], padding="max_length", truncation=True, max_length=128)

# Apply tokenization
train_dataset = train_texts.map(tokenize_function, batched=True)
test_dataset = test_texts.map(tokenize_function, batched=True)


# Set format for PyTorch
train_dataset.set_format(type="torch", columns=["input_ids", "attention_mask", "label"])
test_dataset.set_format(type="torch", columns=["input_ids", "attention_mask", "label"])


/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


Map:   0%|          | 0/2140 [00:00<?, ? examples/s]

Map:   0%|          | 0/536 [00:00<?, ? examples/s]

In [16]:
print(train_texts.column_names)


['Cleaned Text', 'label']


In [17]:
train_dataset[0]

{'label': tensor(2),
 'input_ids': tensor([  101, 16405, 11140,  3022,  2344,  2344,  8909,  2344,  3058,  7661,
          2171,  8746, 11387,  2418,  2692, 20958,  2683,  6285,  2050, 15214,
         21007,  5054,  4031,  4031,  8909,  4031, 24110,  3775,  3775,  3131,
          3976,  2484, 19739, 20486,  2050, 10392,  2050,  1022,  1018,  1019,
          5187,  9300,  2627,  2072,  1019,  3590,  1022,  3931,  1015,   102,
             0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,   

In [18]:
!pip install --upgrade transformers

In [19]:
from transformers import AutoModelForSequenceClassification, TrainingArguments, Trainer

# Load pre-trained BERT model
model = AutoModelForSequenceClassification.from_pretrained(model_name, num_labels=len(label_encoder.classes_))

# Training arguments
# Training arguments
training_args = TrainingArguments(
    output_dir="./results",
    report_to="none",
    eval_strategy="epoch",
    save_strategy="epoch",
    learning_rate=2e-5,
    per_device_train_batch_size=16,
    per_device_eval_batch_size=16,
    num_train_epochs=3,
    weight_decay=0.01,
    load_best_model_at_end=True,
)

# Define metrics
from sklearn.metrics import accuracy_score, precision_recall_fscore_support

def compute_metrics(eval_pred):
    logits, labels = eval_pred
    predictions = logits.argmax(axis=-1)
    precision, recall, f1, _ = precision_recall_fscore_support(labels, predictions, average="weighted")
    acc = accuracy_score(labels, predictions)
    return {"accuracy": acc, "f1": f1, "precision": precision, "recall": recall}

# Create Trainer
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=test_dataset,
    tokenizer=tokenizer,
    compute_metrics=compute_metrics,
)

# Fine-tune the model
trainer.train()


model.safetensors:   0%|          | 0.00/440M [00:00<?, ?B/s]

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
/tmp/ipython-input-1682497407.py:32: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(


Epoch,Training Loss,Validation Loss,Accuracy,F1,Precision,Recall
1,No log,0.005243,1.000000,1.000000,1.000000,1.000000
2,No log,0.002302,1.000000,1.000000,1.000000,1.000000
3,No log,0.001874,1.000000,1.000000,1.000000,1.000000


TrainOutput(global_step=402, training_loss=0.06577765051998309, metrics={'train_runtime': 181.5641, 'train_samples_per_second': 35.359, 'train_steps_per_second': 2.214, 'total_flos': 422300827054080.0, 'train_loss': 0.06577765051998309, 'epoch': 3.0})

In [20]:
import transformers
print(transformers.__version__)

4.55.0


In [21]:
import os

model_save_path = "bert_company_model"
os.makedirs(model_save_path, exist_ok=True)

model.save_pretrained(model_save_path)
tokenizer.save_pretrained(model_save_path)


('bert_company_model/tokenizer_config.json',
 'bert_company_model/special_tokens_map.json',
 'bert_company_model/vocab.txt',
 'bert_company_model/added_tokens.json',
 'bert_company_model/tokenizer.json')

In [22]:
import shutil

shutil.make_archive('bert_company_model', 'zip', model_save_path)


'/content/bert_company_model.zip'

In [23]:
from google.colab import files

files.download('bert_company_model.zip')


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [24]:
import pickle

with open("label_encoder.pkl", "wb") as f:
    pickle.dump(label_encoder, f)


In [25]:
from google.colab import files

files.download("label_encoder.pkl")

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [34]:
!ls /content/

bert_company_model	company-document-text.csv  results
bert_company_model.zip	label_encoder.pkl	   sample_data


In [35]:
import zipfile
import os
from transformers import AutoTokenizer, AutoModelForSequenceClassification
import torch
import pickle

# 1. Unzip the model folder
with zipfile.ZipFile("/content/bert_company_model.zip", 'r') as zip_ref:
    zip_ref.extractall("/content/bert_company_model")

# 2. Set paths
model_path = "/content/bert_company_model"
label_encoder_path = "/content/label_encoder.pkl"

# 3. Load tokenizer and model
tokenizer = AutoTokenizer.from_pretrained(model_path)
model = AutoModelForSequenceClassification.from_pretrained(model_path)

# 4. Load label encoder
with open(label_encoder_path, "rb") as f:
    label_encoder = pickle.load(f)

# 5. Dummy clean_text function (replace if needed)
def clean_text(text):
    return text.strip()

# 6. Prediction function
def predict(text):
    text = clean_text(text)
    inputs = tokenizer(text, return_tensors="pt", padding=True, truncation=True, max_length=128)
    with torch.no_grad():
        outputs = model(**inputs)
        logits = outputs.logits
        predicted_class = torch.argmax(logits, dim=1).item()
    return label_encoder.inverse_transform([predicted_class])[0]

# 7. Example usage
input_text = "invoice order id 10707 customer id arout order date 2017-10-16 ..."
print(predict(input_text))


purchase Order


In [36]:
# Example usage
input_text = "order id  10345 shipping details  ship name  quick-stop ship address  taucherstraße 10 ship city  cunewalde ship region  western europe ship postal code  1307 ship country  germany customer details  customer id  quick customer name  quick-stop employee details  employee name  andrew fuller shipper details  shipper id  2 shipper name  united package order details  order date  2016-11-04 shipped date  2016-11-11 products  -------------------------------------------------------------------------------------------------- product  northwoods cranberry sauce quantity  70 unit price  32 0 total  2240 0 -------------------------------------------------------------------------------------------------- product  teatime chocolate biscuits quantity  80 unit price  7 3 total  584 0 -------------------------------------------------------------------------------------------------- product  singaporean hokkien fried mee quantity  9 unit price  11 2 total  100 8 total price  total price  2924 8"
print(predict(input_text))

ShippingOrder
